In [3]:
from lxml import etree
import pandas as pd
import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker
import sys
import os
sys.path.insert(0,'arxiv.py/')
import arxiv
import databases.create_db_define_models as cre

with open('data/arXiv_src_manifest_Oct_2019.xml', 'r') as f:
    mani = etree.parse(f)
    
# Get list of filenames with xpath
fname = mani.xpath('.//filename')
len(fname)
fname_set = set([f.text for f in fname])


In [4]:
%load_ext autoreload
%autoreload 2
import process as pro

In [11]:
# Connect to the database
database = 'sqlite:///../db_test.db'
downloaded_tars = '/mnt/arXiv_src/downloaded_log.csv'
eng = sa.create_engine(database, echo=False)
eng.connect()
SMaker = sessionmaker(bind=eng)
sess = SMaker()
# don't need this unless creating database from scratch
cre.ManifestTarFile.metadata.create_all(eng)
cre.Article.metadata.create_all(eng)

# Get tarfile set of names
q = sess.query(cre.ManifestTarFile)
filename_set = set([f.filename for f in q.all()])

# Get set of files that have not been fetched for metadata
downloaded_df = pd.read_csv(downloaded_tars)
downloaded_set = set(downloaded_df.filename)
# for some reason src/arXiv_src_manifest.xml is showing up
diff_set = downloaded_set.difference(filename_set)\
                         .difference(set(['src/arXiv_src_manifest.xml']))

len(diff_set)

2498

In [10]:
# Get the name of the file that appears in the manifest file
# Ex. src/arXiv_src_sdf23424.tar
manifest_name = '/'.join(x.tar_path.split('/')[-2:])
MM = mani.xpath('.//*[filename[text()="%s"]]'%manifest_name)[0]
line = parse_element(MM)
f = cre.ManifestTarFile(**line)
sess.add(f)
sess.commit()
x.save_articles_to_db('sqlite:///../db_test.db')

In [9]:
root = mani.getroot()
def parse_element(elem):
    """ return dictionary with manifest metadata """
    return_dict = {}
    for e in elem:
        return_dict[e.tag] = e.text
    return return_dict
def parse_root(root):
        return [parse_element(child) for child in iter(root) if child.tag != 'timestamp']

filedf = pd.DataFrame(parse_root(root))
filedf[['num_items', 'size']] = filedf[['num_items', 'size']].astype(int)
filedf['filename'] = filedf['filename'].astype(str)

In [5]:
mini = pro.Xtraction('tests/minitest3.tar')

querying the arxiv initest3.tar    querying from 00 to 06 successful       

In [7]:
mini.save_articles_to_db('sqlite:///tests/test.db')

In [57]:
for q in mini.query_results:
    print(type(q['tags']),"---", q['arxiv_primary_category'])
    print('')

<class 'str'> --- {'term': 'math.AP', 'scheme': 'http://arxiv.org/schemas/atom'}

<class 'str'> --- {'term': 'math.AG', 'scheme': 'http://arxiv.org/schemas/atom'}

<class 'str'> --- {'term': 'math.AT', 'scheme': 'http://arxiv.org/schemas/atom'}

<class 'str'> --- {'term': 'math.LO', 'scheme': 'http://arxiv.org/schemas/atom'}

<class 'str'> --- {'term': 'math.AP', 'scheme': 'http://arxiv.org/schemas/atom'}

<class 'str'> --- {'term': 'math.NT', 'scheme': 'http://arxiv.org/schemas/atom'}

<class 'str'> --- {'term': 'math.GM', 'scheme': 'http://arxiv.org/schemas/atom'}

<class 'str'> --- {'term': 'math.AP', 'scheme': 'http://arxiv.org/schemas/atom'}

<class 'str'> --- {'term': 'math.DS', 'scheme': 'http://arxiv.org/schemas/atom'}



In [14]:
mini.art_lst

['1403/1403.0035.pdf',
 '1403/1403.0040.gz',
 '1403/1403.0038.gz',
 '1403/1403.0039.gz',
 '1403/1403.0037.gz',
 '1403/1403.0036.pdf']

In [18]:
eng = sa.create_engine('sqlite:///tests/test.db', echo=True)
eng.connect()
SMaker = sessionmaker(bind=eng)
sess = SMaker()
#cre.ManifestTarFile.metadata.create_all(eng)
#cre.Article.metadata.create_all(eng)

2020-02-09 13:46:38,604 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-02-09 13:46:38,607 INFO sqlalchemy.engine.base.Engine ()
2020-02-09 13:46:38,610 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-02-09 13:46:38,612 INFO sqlalchemy.engine.base.Engine ()


In [40]:
sess.query(cre.Article).join(cre.ManifestTarFile).filter(cre.ManifestTarFile.id == 3).all()

2020-02-08 21:22:15,215 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-08 21:22:15,226 INFO sqlalchemy.engine.base.Engine SELECT articles.pk AS articles_pk, articles.tarfile_id AS articles_tarfile_id, articles.id AS articles_id, articles.guidislink AS articles_guidislink, articles.updated_parsed AS articles_updated_parsed, articles.published_parsed AS articles_published_parsed, articles.title AS articles_title, articles.summary AS articles_summary, articles.authors AS articles_authors, articles.author_detail AS articles_author_detail, articles.author AS articles_author, articles.arxiv_comment AS articles_arxiv_comment, articles.tags AS articles_tags, articles.arxiv_url AS articles_arxiv_url, articles.doi AS articles_doi 
FROM articles JOIN manifest ON manifest.id = articles.tarfile_id 
WHERE manifest.id = ?
2020-02-08 21:22:15,228 INFO sqlalchemy.engine.base.Engine (3,)


In [19]:
sess.query(cre.Article).filter(cre.Article.tarfile_id == 5).delete(synchronize_session='fetch')
sess.commit()
sess.query(cre.Article).filter(cre.Article.tarfile_id == 5).all()

2020-02-09 13:46:50,650 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-09 13:46:50,660 INFO sqlalchemy.engine.base.Engine SELECT articles.pk AS articles_pk 
FROM articles 
WHERE articles.tarfile_id = ?
2020-02-09 13:46:50,669 INFO sqlalchemy.engine.base.Engine (5,)
2020-02-09 13:46:50,677 INFO sqlalchemy.engine.base.Engine DELETE FROM articles WHERE articles.tarfile_id = ?
2020-02-09 13:46:50,687 INFO sqlalchemy.engine.base.Engine (5,)
2020-02-09 13:46:50,712 INFO sqlalchemy.engine.base.Engine COMMIT
2020-02-09 13:46:50,890 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-09 13:46:50,895 INFO sqlalchemy.engine.base.Engine SELECT articles.pk AS articles_pk, articles.tarfile_id AS articles_tarfile_id, articles.id AS articles_id, articles.guidislink AS articles_guidislink, articles.updated_parsed AS articles_updated_parsed, articles.published_parsed AS articles_published_parsed, articles.title AS articles_title, articles.summary AS articles_summary, articles.author

[]

In [19]:
pks = sess.query(cre.Article.pk).join(cre.ManifestTarFile).filter(cre.ManifestTarFile.filename.like("src/arXiv_src_133%"))
pks.all()

2020-01-31 21:46:01,530 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-01-31 21:46:01,537 INFO sqlalchemy.engine.base.Engine SELECT articles.pk AS articles_pk 
FROM articles JOIN manifest ON manifest.id = articles.tarfile_id 
WHERE manifest.filename LIKE ?
2020-01-31 21:46:01,543 INFO sqlalchemy.engine.base.Engine ('src/arXiv_src_133%',)


[]

In [190]:
# POPULATE THE MANIFEST TABLE (Careful)
#for index, line in filedf.iterrows():
    #line = filedf.iloc[50*k]
    f = cre.ManifestTarFile(**line)
    sess.add(f)
    sess.commit()

2019-02-03 19:54:26,751 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-02-03 19:54:26,757 INFO sqlalchemy.engine.base.Engine INSERT INTO manifest (filename, content_md5sum, md5sum, first_item, last_item, num_items, seq_num, size, timestamp, yymm) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2019-02-03 19:54:26,762 INFO sqlalchemy.engine.base.Engine ('src/arXiv_src_0001_001.tar', 'cacbfede21d5dfef26f367ec99384546', '949ae880fbaf4649a485a8d9e07f370b', 'astro-ph0001001', 'quant-ph0001119', 2364, '1', 225605507, '2010-12-23 00:13:59', '0001')
2019-02-03 19:54:26,765 INFO sqlalchemy.engine.base.Engine COMMIT
2019-02-03 19:54:26,895 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-02-03 19:54:26,897 INFO sqlalchemy.engine.base.Engine INSERT INTO manifest (filename, content_md5sum, md5sum, first_item, last_item, num_items, seq_num, size, timestamp, yymm) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2019-02-03 19:54:26,904 INFO sqlalchemy.engine.base.Engine ('src/arXiv_src_0002_001.tar', '

KeyboardInterrupt: 

In [227]:
D = arxiv.query(id_list=['1901.009'])[0]
g = cre.new_article_register(D,2)
#sess.add(g)
#sess.commit()

Exception: HTTP Error 400 in query

In [35]:
q = sess.query(cre.ManifestTarFile)
resu = q.filter(cre.ManifestTarFile.num_items > 3000)
resu_set = set([f.filename for f in q.all()])

2020-01-31 23:32:41,487 INFO sqlalchemy.engine.base.Engine SELECT manifest.id AS manifest_id, manifest.filename AS manifest_filename, manifest.content_md5sum AS manifest_content_md5sum, manifest.md5sum AS manifest_md5sum, manifest.first_item AS manifest_first_item, manifest.last_item AS manifest_last_item, manifest.num_items AS manifest_num_items, manifest.seq_num AS manifest_seq_num, manifest.size AS manifest_size, manifest.timestamp AS manifest_timestamp, manifest.yymm AS manifest_yymm 
FROM manifest
2020-01-31 23:32:41,500 INFO sqlalchemy.engine.base.Engine ()


In [56]:
diff_set = fname_set.difference(resu_set)

In [52]:
diff_set.pop()

'src/arXiv_src_1901_012.tar'

In [55]:
len(diff_set)

0

In [54]:
while diff_set:
    diff_set.pop()